<a href="https://colab.research.google.com/github/Danish-Hussain-DSAI/apply-fisherface-on-kalman-filter-images/blob/main/fisherface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score



In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:

# Set the paths to the real and fake image folders
real_folder = "/content/drive/MyDrive/kalman filter/kalman real images"
fake_folder = "/content/drive/MyDrive/kalman filter/kalman fake images"

# Initialize lists to store images and labels



X = []
Y = []


# Load real images and assign the label 'real'
for filename in os.listdir(real_folder):
    img = cv2.imread(os.path.join(real_folder, filename))
    print(img)
    X.append(img)
    Y.append(1)


# Load fake images and assign the label 'fake'
for filename in os.listdir(fake_folder):
    img = cv2.imread(os.path.join(fake_folder, filename))
    print(img)
    X.append(img)
    Y.append(0)

# Convert lists to numpy arrays
X = np.array(X)
Y = np.array(Y)


Streaming output truncated to the last 5000 lines.
  [164 164 164]
  [160 160 160]]]
[[[197 197 197]
  [202 202 202]
  [200 200 200]
  ...
  [122 122 122]
  [114 114 114]
  [112 112 112]]

 [[170 170 170]
  [186 186 186]
  [195 195 195]
  ...
  [127 127 127]
  [116 116 116]
  [116 116 116]]

 [[164 164 164]
  [185 185 185]
  [196 196 196]
  ...
  [142 142 142]
  [124 124 124]
  [119 119 119]]

 ...

 [[174 174 174]
  [214 214 214]
  [233 233 233]
  ...
  [136 136 136]
  [143 143 143]
  [145 145 145]]

 [[180 180 180]
  [189 189 189]
  [180 180 180]
  ...
  [127 127 127]
  [142 142 142]
  [143 143 143]]

 [[182 182 182]
  [177 177 177]
  [127 127 127]
  ...
  [148 148 148]
  [159 159 159]
  [136 136 136]]]
[[[ 66  66  66]
  [ 61  61  61]
  [ 60  60  60]
  ...
  [109 109 109]
  [116 116 116]
  [120 120 120]]

 [[ 81  81  81]
  [ 72  72  72]
  [ 66  66  66]
  ...
  [119 119 119]
  [ 99  99  99]
  [110 110 110]]

 [[ 80  80  80]
  [ 69  69  69]
  [ 63  63  63]
  ...
  [181 181 181]
  [144 

In [24]:
#normalization
X=X/255.0

print(X)

[[[[0.71764706 0.71764706 0.71764706]
   [0.69019608 0.69019608 0.69019608]
   [0.69803922 0.69803922 0.69803922]
   ...
   [0.90196078 0.90196078 0.90196078]
   [0.90196078 0.90196078 0.90196078]
   [0.90196078 0.90196078 0.90196078]]

  [[0.76470588 0.76470588 0.76470588]
   [0.7254902  0.7254902  0.7254902 ]
   [0.71764706 0.71764706 0.71764706]
   ...
   [0.90196078 0.90196078 0.90196078]
   [0.90196078 0.90196078 0.90196078]
   [0.90196078 0.90196078 0.90196078]]

  [[0.78823529 0.78823529 0.78823529]
   [0.74117647 0.74117647 0.74117647]
   [0.71764706 0.71764706 0.71764706]
   ...
   [0.90196078 0.90196078 0.90196078]
   [0.90196078 0.90196078 0.90196078]
   [0.90196078 0.90196078 0.90196078]]

  ...

  [[0.60392157 0.60392157 0.60392157]
   [0.62745098 0.62745098 0.62745098]
   [0.63529412 0.63529412 0.63529412]
   ...
   [0.53333333 0.53333333 0.53333333]
   [0.53333333 0.53333333 0.53333333]
   [0.53333333 0.53333333 0.53333333]]

  [[0.60784314 0.60784314 0.60784314]
   [0.6

In [25]:
grayscale_images = np.mean(X, axis=3)  # Compute the mean along the channels axis

# Common size to which we want to resize the images
target_height, target_width = 128, 128

# Initialize an empty array to store resized images
resized_images = np.empty((X.shape[0], target_height, target_width))

# Resize each image to the target size
for i in range(X.shape[0]):
    resized_images[i] = cv2.resize(grayscale_images[i], (target_width, target_height))


In [26]:
X=np.array(resized_images)
print(X[0])

[[0.7245098  0.70784314 0.70196078 ... 0.9        0.90196078 0.90196078]
 [0.75980392 0.70980392 0.70392157 ... 0.9        0.90196078 0.90196078]
 [0.75686275 0.71176471 0.70686275 ... 0.90196078 0.90196078 0.90196078]
 ...
 [0.60490196 0.6245098  0.62254902 ... 0.53529412 0.52745098 0.5245098 ]
 [0.61372549 0.63333333 0.63137255 ... 0.53921569 0.53235294 0.53137255]
 [0.61862745 0.63823529 0.63823529 ... 0.55       0.54901961 0.54607843]]


In [27]:
print("number of input images : ",np.shape(X))
print("number of labels : ",np.shape(Y))

number of input images :  (200, 128, 128)
number of labels :  (200,)


In [28]:
from sklearn.utils import shuffle

# Shuffle the data while keeping labels synchronized
X, Y = shuffle(X, Y, random_state=42)


In [29]:

# Step 1: Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [30]:
# Step 2: Apply Fisherface for feature extraction
# Assuming X_train and X_test are preprocessed grayscale images

# Initialize the Fisherfaces recognizer
recognizer = cv2.face.FisherFaceRecognizer_create()

In [31]:
# Train the recognizer using the training data
recognizer.train(X_train, Y_train)

In [32]:
# Step 4: Predict labels for the test set
predictions = []
for image in X_test:
    label, _ = recognizer.predict(image)
    predictions.append(label)

# Convert predictions to numpy array
predictions = np.array(predictions)

In [33]:
# Step 5: Train a KNN classifier using the features extracted by Fisherface
# Initialize the KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)

In [34]:
# Flatten the training images
X_train_flat = X_train.reshape(X_train.shape[0], -1)

In [35]:
# Train the KNN classifier
knn.fit(X_train_flat, Y_train)

KNeighborsClassifier()

In [36]:
# Flatten the test images
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [37]:
# Predict using the trained KNN classifier
Y_pred = knn.predict(X_test_flat)

In [38]:
# Step 6: Calculate and display the accuracy
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy of images:", accuracy)

Accuracy: 0.6
